In [1]:
import pandas as pd
df=pd.read_csv('manhattan.csv')
df.head()

,rental_id,rent,bedrooms,bathrooms,size_sqft,min_to_subway,floor,building_age_yrs,no_fee,has_roofdeck,has_washer_dryer,has_doorman,has_elevator,has_dishwasher,has_patio,has_gym,neighborhood,borough
0,1545,2550,0.0,1,480,9,2.0,17,1,1,0,0,1,1,0,1,Upper East Side,Manhattan
1,2472,11500,2.0,2,2000,4,1.0,96,0,0,0,0,0,0,0,0,Greenwich Village,Manhattan
2,2919,4500,1.0,1,916,2,51.0,29,0,1,0,1,1,1,0,0,Midtown,Manhattan
3,2790,4795,1.0,1,975,3,8.0,31,0,0,0,1,1,1,0,1,Greenwich Village,Manhattan
4,3946,17500,2.0,2,4800,3,4.0,136,0,0,0,1,1,1,0,1,Soho,Manhattan


In [2]:
#Feature Scaling 
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df_scaled=df
df_scaled[['rent','size_sqft','min_to_subway','building_age_yrs']]=scaler.fit_transform(df[['rent','size_sqft','min_to_subway','building_age_yrs']])


In [3]:
df_scaled

,rental_id,rent,bedrooms,bathrooms,size_sqft,min_to_subway,floor,building_age_yrs,no_fee,has_roofdeck,has_washer_dryer,has_doorman,has_elevator,has_dishwasher,has_patio,has_gym,neighborhood,borough
0,1545,-0.818669,0.0,1,-0.962011,0.730862,2.0,-0.888763,1,1,0,0,1,1,0,1,Upper East Side,Manhattan
1,2472,2.011480,2.0,2,2.218694,-0.176116,1.0,1.117593,0,0,0,0,0,0,0,0,Greenwich Village,Manhattan
2,2919,-0.202044,1.0,1,-0.049651,-0.538908,51.0,-0.584000,0,1,0,1,1,1,0,0,Midtown,Manhattan
3,2790,-0.108760,1.0,1,0.073811,-0.357512,8.0,-0.533206,0,0,0,1,1,1,0,1,Greenwich Village,Manhattan
4,3946,3.908786,2.0,2,8.077886,-0.357512,4.0,2.133470,0,0,0,1,1,1,0,1,Soho,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,7582,-0.293747,1.0,1,-0.853197,-0.357512,8.0,-0.914160,1,1,1,1,1,1,0,1,Chelsea,Manhattan
3535,5686,0.485729,2.0,2,0.101014,0.005279,10.0,-1.091938,1,1,1,1,1,1,0,1,Tribeca,Manhattan
3536,9679,-1.087770,0.0,1,-1.443302,-0.538908,5.0,1.117593,0,0,0,0,0,0,0,0,Little Italy,Manhattan
3537,5188,-0.526167,1.0,1,-0.604182,0.186675,5.0,-0.964953,1,0,1,1,1,1,0,1,Midtown West,Manhattan


In [4]:
#normal equation method 
import numpy as np
ones=np.ones(3539)
df_scaled_mat=np.asarray(df_scaled)

In [5]:
X=np.column_stack((ones,df_scaled_mat[:,(4,5,7)]))

In [6]:
Y=df_scaled_mat[:,1]

In [7]:
XtX=np.matmul(np.transpose(X),X)
XtX=XtX.astype(float)
XtX_inv=np.linalg.inv(XtX)

In [8]:
XtY=np.matmul(np.transpose(X),Y)
XtY=XtY.astype(float)
theta_hat=np.matmul(XtX_inv,XtY)

In [9]:
theta_hat

array([-5.19716328e-17,  8.61088429e-01, -2.57929494e-02, -1.46134332e-01])

In [10]:
#Check with modules
from sklearn.linear_model import LinearRegression
mlr=LinearRegression()
mlr.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
print(mlr.coef_)

[ 0.          0.86108843 -0.02579295 -0.14613433]


In [12]:
import matplotlib.pyplot as plt
my_apartment=[[1,0.5,0.5,0.5]]
y_predicted=mlr.predict(my_apartment)

In [13]:
y_predicted

array([0.34458057])

In [49]:
#Gradient Descent 
import random
lr=0.01 ; batch=10
theta=[random.uniform(-10.0,10.0) for i in range(4)]
for i in range(int((len(df_scaled)+1)/batch)):

    hx=theta[0]*batch+theta[1]*df_scaled.iloc[batch*(i+1)-10:batch*(i+1),4].sum()+theta[2]*df_scaled.iloc[batch*(i+1)-10:batch*(i+1),5].sum()+theta[3]*df_scaled.iloc[batch*(i+1)-10:batch*(i+1),7].sum()
    y=df_scaled.iloc[batch*(i+1)-10:batch*(i+1),1].sum()
    hxy=hx-y
    theta_new=[0,0,0,0]
    theta_new[0]=theta[0]-lr*hxy/batch
    theta_new[1]=theta[1]-lr*hxy*df_scaled.iloc[batch*(i+1)-10:batch*(i+1),4].sum()/batch
    theta_new[2]=theta[2]-lr*hxy*df_scaled.iloc[batch*(i+1)-10:batch*(i+1),5].sum()/batch
    theta_new[3]=theta[3]-lr*hxy*df_scaled.iloc[batch*(i+1)-10:batch*(i+1),7].sum()/batch
    theta=theta_new

print('Final coefficient: ',theta)
    

Final coefficient:  [0.08969176346963285, 1.0595953024661133, -0.17050173194331406, -0.2815544967301545]


In [55]:
#stochastic gradient descent
lr=0.01 ; batch=1
theta=[random.uniform(-10.0,10.0) for i in range(4)]
for i in range(int((len(df_scaled))/batch)):

    hx=theta[0]*batch+theta[1]*df_scaled.iloc[batch*i,4]+theta[2]*df_scaled.iloc[batch*i,5]+theta[3]*df_scaled.iloc[batch*i,7]
    y=df_scaled.iloc[batch*i,1]
    hxy=hx-y
    theta_new=[0,0,0,0]
    theta_new[0]=theta[0]-lr*hxy/batch
    theta_new[1]=theta[1]-lr*hxy*df_scaled.iloc[batch*i,4]/batch
    theta_new[2]=theta[2]-lr*hxy*df_scaled.iloc[batch*i,5]/batch
    theta_new[3]=theta[3]-lr*hxy*df_scaled.iloc[batch*i,7]/batch
    theta=theta_new

print('Final coefficient: ',theta)

Final coefficient:  [0.028850037940753345, 0.9115932299756583, -0.008270729084877903, -0.098264589756518]


In [63]:
X_new=df_scaled_mat[:,(4,5,7)]

In [71]:
#check with module
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
#reg=make_pipeline(SGDRegressor(max_iter=3500))
#reg.fit(X_new,Y)
reg= SGDRegressor(loss='squared_loss',alpha=0.01,max_iter=3500, learning_rate='constant')

In [72]:
reg.fit(X_new,Y)

SGDRegressor(alpha=0.01, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='constant', loss='squared_loss', max_iter=3500,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [74]:
reg.coef_

array([ 0.79733855, -0.0421617 , -0.1726252 ])

In [89]:
#Logistic Regression
#first with gradient descent
import math
lr=0.01 ; batch=10
theta=[random.uniform(-10.0,10.0) for i in range(4)]

for j in range(int((len(df_scaled)+1)/batch)-1):
    loss=0
    for i in range((j+1)*batch-10,(j+1)*batch):
        loss=[0,0,0,0]
        thx=theta[0]*batch+theta[1]*df_scaled.iloc[i,4]+theta[2]*df_scaled.iloc[i,5]+theta[3]*df_scaled.iloc[i,7]
        hx=1/(1+math.exp(-thx))
        y=df_scaled.iloc[i,12]
        k=hx-y

        loss[0]+=k
        loss[1]+=k*df_scaled.iloc[i,4]
        loss[2]+=k*df_scaled.iloc[i,5]
        loss[3]+=k*df_scaled.iloc[i,7]
        
    
    theta_new=[0,0,0,0]
    theta_new[0]=theta[0]-lr*(1/batch)*loss[0]
    theta_new[1]=theta[1]-lr*(1/batch)*loss[1]
    theta_new[2]=theta[2]-lr*(1/batch)*loss[2]
    theta_new[3]=theta[3]-lr*(1/batch)*loss[3]
    theta=theta_new
    
print('Final coefficient:',theta)

Final coefficient: [5.094926384961473, -1.965821273138831, -4.020483722661868, -9.307814694884994]


In [92]:
from sklearn.linear_model import LogisticRegression
Y_log=df_scaled.iloc[:,12]
logreg=LogisticRegression(random_state=0, max_iter=3500).fit(X_new,Y_log)

In [93]:
logreg.coef_  # why are 

array([[ 0.09176661, -0.03040091, -0.14218592]])

In [94]:
logreg.intercept_

array([-0.87820341])